In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import SVG

from pydrake.examples.quadrotor import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from models.building_generation import *
from spp.bezier import BezierSPP
from spp.rounding import *

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
def run_planning(start, goal, weights, save_location=None, overlap=False):
    if save_location is not None and not os.path.exists(save_location):
        os.makedirs(save_location)
        
    grid, outdoor_edges, wall_edges = generate_grid_world(shape=(3, 3), start=start, goal=goal)
    file_location = "models/room_gen/building.sdf"
    if save_location is not None:
        file_location = save_location + "/building.sdf"
    regions = compile_sdf(file_location, grid, start, goal, outdoor_edges, wall_edges, overlap=overlap)
    
    order = 7
    continuity = 4
    vel_limit = 10 * np.ones(3)
    start_pose = np.r_[(start-start)*5, 1.]
    goal_pose = np.r_[(goal-start)*5., 1.]
    
    start_setup = time.time()
    b_spp = BezierSPP(regions, order, continuity, hdot_min=1e-3)
    b_spp.addTimeCost(weights["time"])
    b_spp.addPathLengthCost(weights["norm"])
    b_spp.addVelocityLimits(-vel_limit, vel_limit)
    b_spp.setPaperSolverOptions()
    b_spp.setSolver(MosekSolver())
    b_spp.addAccelerationRegularization(1e-3, 1e-3)
    b_spp.setRoundingStrategy([greedyForwardPathSearch, greedyBackwardPathSearch])
    setup_time = time.time() - start_setup
    
    start_relax = time.time()
    relax_output = b_spp.SolvePath(start_pose, goal_pose, True, False, zero_deriv_boundary=3)
    b_traj_relax, result_relax, best_result_relax, hard_result_relax = relax_output
    relax_time = time.time() - start_relax
    
    start_mip = time.time()
    b_traj_mip, result_mip = b_spp.SolvePath(start_pose, goal_pose, False, False, zero_deriv_boundary=3)[:2]
    mip_time = time.time() - start_mip
    
    planning_results = dict()
    planning_results["order"] = order
    planning_results["continuity"] = continuity
    planning_results["velocity_limit"] = vel_limit
    planning_results["start_pose"] = start_pose
    planning_results["goal_pose"] = goal_pose
    
    planning_results["setup_time"] = setup_time
    planning_results["relaxation_time"] = relax_time
    planning_results["mip_time"] = mip_time
    
    planning_results["relaxation_solver_1_result"] = result_relax.get_solution_result()
    planning_results["relaxation_solver_1_time"] = result_relax.get_solver_details().optimizer_time
    planning_results["relaxation_solver_1_cost"] = result_relax.get_optimal_cost()
    planning_results["relaxation_solver_1_solution"] = []
    for edge in b_spp.spp.Edges():
        edge_solution = {"name": edge.name(),
                         "y_e": edge.GetSolutionPhiXu(result_relax),
                         "z_e": edge.GetSolutionPhiXv(result_relax),
                         "phi_e": result_relax.GetSolution(edge.phi())}
        planning_results["relaxation_solver_1_solution"].append(edge_solution)
    
    if best_result_relax is not None:
        planning_results["relaxation_solver_2_result"] = best_result_relax.get_solution_result()
        planning_results["relaxation_solver_2_cost"] = best_result_relax.get_optimal_cost()
        planning_results["relaxation_solver_2_solution"] = []
        for edge in b_spp.spp.Edges():
            edge_solution = {"name": edge.name(),
                             "y_e": edge.GetSolutionPhiXu(best_result_relax),
                             "z_e": edge.GetSolutionPhiXv(best_result_relax),
                             "phi_e": best_result_relax.GetSolution(edge.phi())}
            planning_results["relaxation_solver_2_solution"].append(edge_solution)
        
        max_hard_result_time = 0
        for result in hard_result_relax:
            if result.get_solver_details().optimizer_time > max_hard_result_time:
                max_hard_result_time = result.get_solver_details().optimizer_time
        planning_results["relaxation_solver_2_time"] = max_hard_result_time
        planning_results["relaxation_solver_total_time"] = (planning_results["relaxation_solver_1_time"]
                                                            + planning_results["relaxation_solver_2_time"])
    else:
        planning_results["relaxation_solver_2_result"] = None
        planning_results["relaxation_solver_2_time"] = np.nan
        planning_results["relaxation_solver_total_time"] = np.nan
        planning_results["relaxation_solver_2_cost"] = np.nan
        planning_results["relaxation_solver_2_solution"] = None
    
    planning_results["mip_solver_result"] = result_mip.get_solution_result()
    planning_results["mip_solver_time"] = result_mip.get_solver_details().optimizer_time
    planning_results["mip_solver_cost"] = result_mip.get_optimal_cost()
    planning_results["mip_solver_solution"] = []
    for edge in b_spp.spp.Edges():
        edge_solution = {"name": edge.name(),
                         "y_e": edge.GetSolutionPhiXu(result_mip),
                         "z_e": edge.GetSolutionPhiXv(result_mip),
                         "phi_e": result_mip.GetSolution(edge.phi())}
        planning_results["mip_solver_solution"].append(edge_solution)
    
    print("Solve times:", relax_time, mip_time, flush=True)
    
    if save_location is not None:
        print("Saving files to", save_location, flush=True)
        with open(save_location + '/regions.reg', 'wb') as f:
            pickle.dump(regions, f)
        with open(save_location + "/relaxation_traj.pkl", "wb") as f:
            pickle.dump(b_traj_relax, f, pickle.HIGHEST_PROTOCOL)
        with open(save_location + "/mip_traj.pkl", "wb") as f:
            pickle.dump(b_traj_mip, f, pickle.HIGHEST_PROTOCOL)
        with open(save_location + '/plan_results.pkl', 'wb') as f:
            pickle.dump(planning_results, f)

In [ ]:
start=np.array([-1, -1])
goal=np.array([2, 1])
weights = {"time": 1., "norm": 1.}
np.random.seed(42)
runs = 50
start_time = time.time()
for ii in range(runs):
    run_planning(start, goal, weights, "data/room_gen/smooth_paths/room_" + str(ii).zfill(3), overlap=False)
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4), "minutes.")

In [ ]:
start=np.array([-1, -1])
goal=np.array([2, 1])
weights = {"time": 1., "norm": 1.}
np.random.seed(42)
runs = 50
start_time = time.time()
for ii in range(runs):
    run_planning(start, goal, weights, "data/room_gen/overlap_trees/room_" + str(ii).zfill(3), overlap=True)
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4))

In [ ]:
runs = 2
failed_solves = []

relax_costs = np.empty(runs - len(failed_solves))
rounded_costs = np.empty(runs - len(failed_solves))
mip_costs = np.empty(runs - len(failed_solves))
relaxation_solver_time = np.empty(runs - len(failed_solves))
relaxation_time = np.empty(runs - len(failed_solves))
mip_solver_time = np.empty(runs - len(failed_solves))
mip_time = np.empty(runs - len(failed_solves))

ii = 0
for index in range(runs):
    if index in failed_solves:
        continue
    save_location = "data/room_gen/smooth_paths/room_" + str(index).zfill(3)
    with open(save_location + '/plan_results.pkl', "rb") as f:
        data = pickle.load(f)
        relax_costs[ii] = data["relaxation_solver_1_cost"]
        rounded_costs[ii] = data["relaxation_solver_2_cost"]
        mip_costs[ii] = data["mip_solver_cost"]
        relaxation_solver_time[ii] = data["relaxation_solver_total_time"]
        relaxation_time[ii] = data["relaxation_time"]
        mip_solver_time[ii] = data["mip_solver_time"]
        mip_time[ii] = data["mip_time"]
        
    ii += 1
    
rounding_gap = (rounded_costs-relax_costs)/relax_costs
solution_gap = (rounded_costs-mip_costs)/mip_costs
relaxation_gap = (mip_costs - relax_costs)/mip_costs

optimality_tolerance = 0.01

print("Tight Rounding:", np.sum(rounding_gap < optimality_tolerance)/ii)
print("Mean rounding gap:", np.mean(rounding_gap))
print("Max rounding gap:", np.argmax(rounding_gap), np.max(rounding_gap))
print()

print("Tight relaxation:", np.sum(relaxation_gap < optimality_tolerance)/ii)
print("Mean relaxation gap:", np.mean(relaxation_gap))
print("Max relaxation gap:", np.argmax(relaxation_gap), np.max(relaxation_gap))
print()

print("Solved to opimality:", np.sum(solution_gap < optimality_tolerance)/ii)
print("Mean solution gap:", np.mean(solution_gap))
print("Max solution gap:", np.argmax(solution_gap), np.max(solution_gap))

print()
print("Mean Rounding Solver Time:", np.mean(relaxation_solver_time))
print("Mean MIP Solver Time:", np.mean(mip_solver_time))
print("Mean MIP/Rounding Solver Time:", np.mean(mip_solver_time/relaxation_solver_time))


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [ ]:
start=np.array([-1, -1])
goal=np.array([2, 1])
grid, indoor_edges, wall_edges = generate_grid_world(shape=(3,3), start=start, goal=goal, seed=42)
# draw_grid_world(grid, start, goal, indoor_edges, wall_edges)

regions = compile_sdf("models/room_gen/building.sdf", grid, start, goal, indoor_edges, wall_edges, seed=42)

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()

MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)

# if running_as_notebook:
simulator.set_target_realtime_rate(1.0)

# Set the initial conditions
context = simulator.get_mutable_context()

meshcat.Delete()
for ii in range(len(regions)):
    v = VPolytope(regions[ii])
    meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                            ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))

# Simulate
simulator.AdvanceTo(0.1)

In [ ]:
class FlatnessInverter(LeafSystem):
    def __init__(self, traj, animator, t_offset=0):
        LeafSystem.__init__(self)
        self.traj = traj
        self.port = self.DeclareVectorOutputPort("state", 12, self.DoCalcState, {self.time_ticket()})
        self.t_offset = t_offset
        self.animator = animator
        
    def DoCalcState(self, context, output):
        t = context.get_time() + self.t_offset - 1e-4
        
        q = np.squeeze(self.traj.value(t))
        q_dot = np.squeeze(self.traj.EvalDerivative(t))
        q_ddot = np.squeeze(self.traj.EvalDerivative(t, 2))
        
        fz = np.sqrt(q_ddot[0]**2 + q_ddot[1]**2 + (q_ddot[2] + 9.81)**2)
        r = np.arcsin(-q_ddot[1]/fz)
        p = np.arcsin(q_ddot[0]/fz)
        
        output.set_value(np.concatenate((q, [r, p, 0], q_dot, np.zeros(3))))
        
        if self.animator is not None:
            frame = animator.frame(context.get_time())
            animator.SetProperty(frame, "/Cameras/default/rotated/<object>", "position", [-2.5, 4, 2.5])
            animator.SetTransform(frame, "/drake", RigidTransform(-q))

In [ ]:
view_relaxation = False
view_regions = False
track_uav = True
save_html = False
room = 9

save_location = "data/room_gen/smooth_paths/room_" + str(room).zfill(3)

# Load data from disk
shutil.copy(save_location + "/building.sdf", "models/room_gen/building.sdf")

if view_regions:
    with open(save_location + "/regions.reg", "rb") as f:
        regions = pickle.load(f)

if view_relaxation:
    with open(save_location + "/relaxation_traj.pkl", "rb") as f:
        b_traj = pickle.load(f)
else:
    with open(save_location + "/mip_traj.pkl", "rb") as f:
        b_traj = pickle.load(f)

# Build and run Diagram
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()

meshcat_cpp = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

if b_traj is not None:
    animator = meshcat_cpp.StartRecording()
    if not track_uav:
        animator = None
    traj_system = builder.AddSystem(FlatnessInverter(b_traj, animator))
    quad = QuadrotorGeometry.AddToBuilder(builder, traj_system.get_output_port(0), scene_graph)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

meshcat.Delete()

if view_regions:
    for ii in range(len(regions)):
        v = VPolytope(regions[ii])
        meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                                ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))

if track_uav:
    meshcat.SetProperty("/Grid", "visible", False)
    meshcat.SetProperty("/Axes", "visible", False)
        
# Simulate
if b_traj is not None:
    end_time = b_traj.end_time()
    simulator.AdvanceTo(end_time+0.05)
    meshcat_cpp.PublishRecording()
    
    if save_html:
        with open ("data/uav_trajectory.html", "w") as f:
           f.write(meshcat.StaticHtml())
else:
    simulator.AdvanceTo(0.1)